<a href="https://colab.research.google.com/github/hareshbaskaran/Sarcasm-on-Reddit/blob/master/sarcasm_detect_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import os
for dirname, _, filenames in os.walk('/kaggle'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [2]:
train_df= pd.read_csv("/content/drive/MyDrive/train-balanced-sarcasm.csv",)
train_df.head()

FileNotFoundError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
len(train_df),train_df.index.shape[-1]


In [ ]:
import numpy as np
import pandas as pd
from sklearn import preprocessing,metrics,manifold
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV,cross_val_predict
from imblearn.over_sampling import ADASYN,SMOTE
from imblearn.under_sampling import NearMiss
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
import collections
import matplotlib.patches as mpatches
from sklearn.metrics import accuracy_score
%matplotlib inline
from sklearn.preprocessing import RobustScaler
import xgboost
from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import classification_report,roc_auc_score,roc_curve,r2_score,recall_score,confusion_matrix,precision_recall_curve
from collections import Counter
from sklearn.model_selection import StratifiedKFold,KFold,StratifiedShuffleSplit
from nltk import word_tokenize
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA, TruncatedSVD,SparsePCA
from sklearn.metrics import classification_report,confusion_matrix
from nltk.tokenize import word_tokenize
from collections import defaultdict
from collections import Counter
import seaborn as sns
from wordcloud import WordCloud,STOPWORDS
import nltk
from nltk.corpus import stopwords
import string
from plotly import tools
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import keras
from keras.preprocessing.text import Tokenizer
import tensorflow as tf
from keras.layers import Dense, BatchNormalization
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [ ]:
print("Sntiment - ".format(),train_df.shape)

In [ ]:
train_df = train_df.dropna()
train_df.info()

In [ ]:
dict_df={
    "label":train_df["label"],
    "comment":train_df["comment"],
}
df = pd.DataFrame(dict_df)
df["comment"] = df["comment"].astype(str)
df.head()

In [ ]:
#lowercasing 
df["text_lower"] = df["comment"].str.lower()
df.head(10)

In [ ]:
#punctuation removal
# PUNCT_TO_REMOVE = string.punctuation
# def remove_punctuation(text):
#     """custom function to remove the punctuation"""
#     return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

# df["text_wo_punct"] = df["text_lower"].apply(lambda text: remove_punctuation(text))
# df.head()

In [ ]:
#stopword removal
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
", ".join(stopwords.words('english'))

In [ ]:
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

df["text_wo_stop"] = df["text_lower"].apply(lambda text: remove_stopwords(text))
df.head()

In [ ]:
#removal of frequent words
# from collections import Counter
# cnt = Counter()
# for text in df["text_wo_stop"].values:
#     for word in text.split():
#         cnt[word] += 1
        
# cnt.most_common(20)

In [ ]:
# FREQWORDS = set([w for (w, wc) in cnt.most_common(10)])
# def remove_freqwords(text):
#     """custom function to remove the frequent words"""
#     return " ".join([word for word in str(text).split() if word not in FREQWORDS])

# df["text_wo_stopfreq"] = df["text_wo_stop"].apply(lambda text: remove_freqwords(text))
# df.head()

In [ ]:
#Reduction of inflected words Methods
#STEMMING
# from nltk.stem.porter import PorterStemmer

# # Drop the two columns 

# stemmer = PorterStemmer()
# def stem_words(text):
#     return " ".join([stemmer.stem(word) for word in text.split()])

# df["text_stemmed"] = df["text_wo_stop"].apply(lambda text: stem_words(text))
# df.head()


In [ ]:
#Snowball stemmer 
# from nltk.stem.snowball import SnowballStemmer
# SnowballStemmer.languages

In [ ]:
#lemmatization
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

df["text_lemmatized"] = df["text_wo_stop"].apply(lambda text: lemmatize_words(text))
df.head()

In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
wordnet_map = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV}
def lemmatize_words(text):
    pos_tagged_text = nltk.pos_tag(text.split())
    return " ".join([lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tagged_text])

df["text_lemmatized"] = df["text_wo_stop"].apply(lambda text: lemmatize_words(text))
df.head()

In [ ]:
df.info()


##Tokenization of Comments

In [ ]:
from nltk.lm.preprocessing import pad_sequence
tokenizer=Tokenizer(num_words=1000,oov_token='<OOV>')
tokenizer.fit_on_texts(df["text_lemmatized"])

In [ ]:
df["tokens"] = tokenizer.texts_to_sequences(df["text_lemmatized"])
df["length"] = df["tokens"].apply(lambda x: len(x))
comments = pad_sequences(df["tokens"])
labels = df["label"]

print(comments)

## Test train split


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(comments, labels, train_size = 0.8, random_state = 42, shuffle = True)


##CNN Model

In [ ]:
cnn_model = keras.Sequential([
    keras.layers.Embedding(50000, 64, input_length=100),
    keras.layers.Conv1D(128, 5, activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.GlobalMaxPooling1D(),
    keras.layers.BatchNormalization(),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

cnn_model.compile(loss='binary_crossentropy',optimizer=tf.keras.optimizers.Adam(0.0002),metrics=['accuracy'])
cnn_model.summary()

In [ ]:
cnn_model.fit(X_train,y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test), verbose=1)


## LSTM Model

In [ ]:
lstm_model = keras.Sequential([
    keras.layers.Embedding(50000, 64, input_length=100),
    keras.layers.Conv1D(128,5,activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),
    keras.layers.Dropout(0.2),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.2),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

lstm_model.compile(loss="binary_crossentropy", optimizer='rmsprop', metrics=["binary_accuracy"])
lstm_model.summary()
